In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Setting
sns.despine(left=True, bottom=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format','{:.4f}'.format)
sns.set()

%matplotlib inline

In [2]:
jams = pd.read_parquet('..\\dataset\\aggregate_median_jams_bogor_eda.parquet')

In [3]:
jams.sample(5)

,street,median_length,median_delay,median_speed_kmh,total_records,id,date,median_level,geometry,date_time,weekday,month,day,is_weekday,hour,is_working_hour,is_morning,is_lunch,is_late_afternoon,is_late_night
36309,siliwangi,668.0000,173.0000,9.3650,26,34880272,2022-07-28,3.0000,"MULTILINESTRING ((106.810679 -6.616666, 106.81...",2022-07-28 13:00:00,3,7,28,1,13,1,0,1,0,0
99235,jalan empang,406.0000,71.0000,11.3200,1,37148338,2022-09-03,2.0000,"LINESTRING (106.793492 -6.607427, 106.793927 -...",2022-09-03 15:00:00,5,9,3,0,15,0,0,0,0,0
17362,n9 kh soleh iskandar,458.0000,140.0000,6.2200,10,34106103,2022-07-16,4.0000,"MULTILINESTRING ((106.807139 -6.561478, 106.80...",2022-07-16 22:00:00,5,7,16,0,22,0,0,0,0,0
90373,surya kencana,1165.0000,127.0000,13.0400,25,36846250,2022-08-29,2.0000,"MULTILINESTRING ((106.799024 -6.603009, 106.79...",2022-08-29 11:00:00,0,8,29,1,11,1,0,0,0,0
59265,siliwangi,556.0000,81.0000,15.0100,13,35716689,2022-08-10,2.0000,"MULTILINESTRING ((106.805577 -6.610952, 106.80...",2022-08-10 12:00:00,2,8,10,1,12,1,0,1,0,0


In [102]:
street_bogor = jams['street'].value_counts().reset_index()
street_bogor = street_bogor.rename(columns={'index':'street', 'street':'occurance'})
street_bogor = street_bogor.merge(jams[['street', 'geometry']], how='left', on='street')
street_bogor = street_bogor.drop_duplicates()
street_bogor.to_parquet('..\\dataset\\street_jams_bogor_preprocess.parquet')

## Clustering Preprocess

devided the dataset into different time span, then feature engineering on them
- (1) early morning (3-6)
- (2) morning (6-10)
- (3) afternoon (10-15)
- (4) late afternoon (15-18)
- (5) night (18-22)
- (6) late night (22-3)

In [11]:
jams['time_span'] = np.where((jams.hour >= 3) & (jams.hour < 6), 1, 
                             np.where((jams.hour >= 6) & (jams.hour < 10), 2,
                                      np.where((jams.hour >= 10) & (jams.hour < 15), 3,
                                               np.where((jams.hour >= 15) & (jams.hour < 18), 4,
                                                        np.where((jams.hour >= 18) & (jams.hour < 22), 5, 6))))
                             )

In [21]:
jams_early_morning = jams.query('time_span == 1')
jams_morning = jams.query('time_span == 2')
jams_afternoon = jams.query('time_span == 3')
jams_late_afternoon = jams.query('time_span == 4')
jams_night = jams.query('time_span == 5')
jams_late_night = jams.query('time_span == 6')

### Feature Engineer: Numerical

In [27]:
temp_df = (
    jams_morning
    .loc[:, ['median_length', 'median_delay', 'median_speed_kmh', 'total_records', 'median_level', 'street']]
    .groupby('street')
    .agg(['mean', 'median', 'std', 'count'])
)

temp_df = temp_df.fillna(0)
temp_df.columns = [f'{col[0]}__{col[1]}' for col in temp_df.columns]

In [93]:
temp_df.sample()

,median_length__mean,median_length__median,median_length__std,median_length__count,median_delay__mean,median_delay__median,median_delay__std,median_delay__count,median_speed_kmh__mean,median_speed_kmh__median,median_speed_kmh__std,median_speed_kmh__count,total_records__mean,total_records__median,total_records__std,total_records__count,median_level__mean,median_level__median,median_level__std,median_level__count
street,,,,,,,,,,,,,,,,,,,,
menteng,492.0000,492.0000,0.0000,1,136.0000,136.0000,0.0000,1,7.2000,7.2000,0.0000,1,7.0000,7.0000,0.0000,1,3.0000,3.0000,0.0000,1


### Feature Engineer `weekday`

In [69]:
temp_df_weekday = (
    jams_morning
    .loc[:, ['median_length', 'median_delay', 'median_speed_kmh', 'total_records', 'median_level', 'street', 'weekday']]
    .groupby(['street', 'weekday'])
    .agg(['mean', 'median', 'std', 'count'])
    )

temp_df_weekday = temp_df_weekday.fillna(0)
temp_df_weekday.columns = [f'{col[0]}__{col[1]}' for col in temp_df_weekday.columns]

In [70]:
temp_df_weekday = temp_df_weekday.reset_index()

In [74]:
value_cols = list(temp_df_weekday.columns)
value_cols.remove('weekday')
value_cols.remove('street')

temp_df_weekday = temp_df_weekday.pivot(index='street',
                                        columns='weekday',
                                        values=value_cols)

In [77]:
temp_df_weekday = temp_df_weekday.fillna(0)
temp_df_weekday.columns = [f'{col[0]}__weekday_{col[1]}' for col in temp_df_weekday.columns]

In [85]:
temp_df_weekday.sample(5)

,median_length__mean__weekday_0,median_length__mean__weekday_1,median_length__mean__weekday_2,median_length__mean__weekday_3,median_length__mean__weekday_4,median_length__mean__weekday_5,median_length__mean__weekday_6,median_length__median__weekday_0,median_length__median__weekday_1,median_length__median__weekday_2,median_length__median__weekday_3,median_length__median__weekday_4,median_length__median__weekday_5,median_length__median__weekday_6,median_length__std__weekday_0,median_length__std__weekday_1,median_length__std__weekday_2,median_length__std__weekday_3,median_length__std__weekday_4,median_length__std__weekday_5,median_length__std__weekday_6,median_length__count__weekday_0,median_length__count__weekday_1,median_length__count__weekday_2,median_length__count__weekday_3,median_length__count__weekday_4,median_length__count__weekday_5,median_length__count__weekday_6,median_delay__mean__weekday_0,median_delay__mean__weekday_1,median_delay__mean__weekday_2,median_delay__mean__weekday_3,median_delay__mean__weekday_4,median_delay__mean__weekday_5,median_delay__mean__weekday_6,median_delay__median__weekday_0,median_delay__median__weekday_1,median_delay__median__weekday_2,median_delay__median__weekday_3,median_delay__median__weekday_4,median_delay__median__weekday_5,median_delay__median__weekday_6,median_delay__std__weekday_0,median_delay__std__weekday_1,median_delay__std__weekday_2,median_delay__std__weekday_3,median_delay__std__weekday_4,median_delay__std__weekday_5,median_delay__std__weekday_6,median_delay__count__weekday_0,median_delay__count__weekday_1,median_delay__count__weekday_2,median_delay__count__weekday_3,median_delay__count__weekday_4,median_delay__count__weekday_5,median_delay__count__weekday_6,median_speed_kmh__mean__weekday_0,median_speed_kmh__mean__weekday_1,median_speed_kmh__mean__weekday_2,median_speed_kmh__mean__weekday_3,median_speed_kmh__mean__weekday_4,median_speed_kmh__mean__weekday_5,median_speed_kmh__mean__weekday_6,median_speed_kmh__median__weekday_0,median_speed_kmh__median__weekday_1,median_speed_kmh__median__weekday_2,median_speed_kmh__median__weekday_3,median_speed_kmh__median__weekday_4,median_speed_kmh__median__weekday_5,median_speed_kmh__median__weekday_6,median_speed_kmh__std__weekday_0,median_speed_kmh__std__weekday_1,median_speed_kmh__std__weekday_2,median_speed_kmh__std__weekday_3,median_speed_kmh__std__weekday_4,median_speed_kmh__std__weekday_5,median_speed_kmh__std__weekday_6,median_speed_kmh__count__weekday_0,median_speed_kmh__count__weekday_1,median_speed_kmh__count__weekday_2,median_speed_kmh__count__weekday_3,median_speed_kmh__count__weekday_4,median_speed_kmh__count__weekday_5,median_speed_kmh__count__weekday_6,total_records__mean__weekday_0,total_records__mean__weekday_1,total_records__mean__weekday_2,total_records__mean__weekday_3,total_records__mean__weekday_4,total_records__mean__weekday_5,total_records__mean__weekday_6,total_records__median__weekday_0,total_records__median__weekday_1,total_records__median__weekday_2,total_records__median__weekday_3,total_records__median__weekday_4,total_records__median__weekday_5,total_records__median__weekday_6,total_records__std__weekday_0,total_records__std__weekday_1,total_records__std__weekday_2,total_records__std__weekday_3,total_records__std__weekday_4,total_records__std__weekday_5,total_records__std__weekday_6,total_records__count__weekday_0,total_records__count__weekday_1,total_records__count__weekday_2,total_records__count__weekday_3,total_records__count__weekday_4,total_records__count__weekday_5,total_records__count__weekday_6,median_level__mean__weekday_0,median_level__mean__weekday_1,median_level__mean__weekday_2,median_level__mean__weekday_3,median_level__mean__weekday_4,median_level__mean__weekday_5,median_level__mean__weekday_6,median_level__median__weekday_0,median_level__median__weekday_1,median_level__median__weekday_2,median_level__median__weekday_3,median_level__median__weekday_4,median_level__median__weekday_5,median_level_

### Feature Engineer `day`

In [83]:
temp_df_day = (
    jams_morning
    .loc[:, ['median_length', 'median_delay', 'median_speed_kmh', 'total_records', 'median_level', 'street', 'day']]
    .groupby(['street', 'day'])
    .agg(['mean', 'median', 'std', 'count'])
    )

temp_df_day = temp_df_day.fillna(0)
temp_df_day.columns = [f'{col[0]}__{col[1]}' for col in temp_df_day.columns]

In [84]:
temp_df_day = temp_df_day.reset_index()

In [86]:
value_cols = list(temp_df_day.columns)
value_cols.remove('day')
value_cols.remove('street')

temp_df_day = temp_df_day.pivot(index='street',
                                columns='day',
                                values=value_cols)

In [87]:
temp_df_day = temp_df_day.fillna(0)
temp_df_day.columns = [f'{col[0]}__day_{col[1]}' for col in temp_df_day.columns]

In [90]:
temp_df_day.sample()

,median_length__mean__day_1,median_length__mean__day_2,median_length__mean__day_3,median_length__mean__day_4,median_length__mean__day_5,median_length__mean__day_6,median_length__mean__day_7,median_length__mean__day_8,median_length__mean__day_9,median_length__mean__day_10,median_length__mean__day_11,median_length__mean__day_12,median_length__mean__day_13,median_length__mean__day_14,median_length__mean__day_15,median_length__mean__day_16,median_length__mean__day_17,median_length__mean__day_18,median_length__mean__day_19,median_length__mean__day_20,median_length__mean__day_21,median_length__mean__day_22,median_length__mean__day_23,median_length__mean__day_24,median_length__mean__day_25,median_length__mean__day_26,median_length__mean__day_27,median_length__mean__day_28,median_length__mean__day_29,median_length__mean__day_30,median_length__mean__day_31,median_length__median__day_1,median_length__median__day_2,median_length__median__day_3,median_length__median__day_4,median_length__median__day_5,median_length__median__day_6,median_length__median__day_7,median_length__median__day_8,median_length__median__day_9,median_length__median__day_10,median_length__median__day_11,median_length__median__day_12,median_length__median__day_13,median_length__median__day_14,median_length__median__day_15,median_length__median__day_16,median_length__median__day_17,median_length__median__day_18,median_length__median__day_19,median_length__median__day_20,median_length__median__day_21,median_length__median__day_22,median_length__median__day_23,median_length__median__day_24,median_length__median__day_25,median_length__median__day_26,median_length__median__day_27,median_length__median__day_28,median_length__median__day_29,median_length__median__day_30,median_length__median__day_31,median_length__std__day_1,median_length__std__day_2,median_length__std__day_3,median_length__std__day_4,median_length__std__day_5,median_length__std__day_6,median_length__std__day_7,median_length__std__day_8,median_length__std__day_9,median_length__std__day_10,median_length__std__day_11,median_length__std__day_12,median_length__std__day_13,median_length__std__day_14,median_length__std__day_15,median_length__std__day_16,median_length__std__day_17,median_length__std__day_18,median_length__std__day_19,median_length__std__day_20,median_length__std__day_21,median_length__std__day_22,median_length__std__day_23,median_length__std__day_24,median_length__std__day_25,median_length__std__day_26,median_length__std__day_27,median_length__std__day_28,median_length__std__day_29,median_length__std__day_30,median_length__std__day_31,median_length__count__day_1,median_length__count__day_2,median_length__count__day_3,median_length__count__day_4,median_length__count__day_5,median_length__count__day_6,median_length__count__day_7,median_length__count__day_8,median_length__count__day_9,median_length__count__day_10,median_length__count__day_11,median_length__count__day_12,median_length__count__day_13,median_length__count__day_14,median_length__count__day_15,median_length__count__day_16,median_length__count__day_17,median_length__count__day_18,median_length__count__day_19,median_length__count__day_20,median_length__count__day_21,median_length__count__day_22,median_length__count__day_23,median_length__count__day_24,median_length__count__day_25,median_length__count__day_26,median_length__count__day_27,median_length__count__day_28,median_length__count__day_29,median_length__count__day_30,median_length__count__day_31,median_delay__mean__day_1,median_delay__mean__day_2,median_delay__mean__day_3,median_delay__mean__day_4,median_delay__mean__day_5,median_delay__mean__day_6,median_delay__mean__day_7,median_delay__mean__day_8,median_delay__mean__day_9,median_delay__mean__day_10,median_delay__mean__day_11,median_delay__mean__day_12,median_delay__mean__day_13,median_delay__mean__day_14,median_delay__mean__day_15,median_delay__mean__day_16,median_delay__mean__day_17,median_delay__mean__day_18,median_delay__mean__day_

## Feature Engineering Pipeline

In [119]:
def feat_eng_numerical(dataframe):
    temp_df = (
        dataframe
        .loc[:, ['median_length', 'median_delay', 'median_speed_kmh', 'total_records', 'median_level', 'street']]
        .groupby('street')
        .agg(['mean', 'median', 'std', 'count'])
    )

    temp_df = temp_df.fillna(0)
    temp_df.columns = [f'{col[0]}__{col[1]}' for col in temp_df.columns]
    return temp_df


def feat_eng_weekday(dataframe):
    temp_df_weekday = (
                        dataframe
                        .loc[:, ['median_length', 'median_delay', 'median_speed_kmh', 'total_records', 'median_level', 'street', 'weekday']]
                        .groupby(['street', 'weekday'])
                        .agg(['mean', 'median', 'std', 'count'])
                        )

    temp_df_weekday = temp_df_weekday.fillna(0)
    temp_df_weekday.columns = [f'{col[0]}__{col[1]}' for col in temp_df_weekday.columns]
    temp_df_weekday = temp_df_weekday.reset_index()
    
    value_cols = list(temp_df_weekday.columns)
    value_cols.remove('weekday')
    value_cols.remove('street')

    temp_df_weekday = temp_df_weekday.pivot(index='street', columns='weekday', values=value_cols)
    temp_df_weekday = temp_df_weekday.fillna(0)
    temp_df_weekday.columns = [f'{col[0]}__weekday_{col[1]}' for col in temp_df_weekday.columns]
    
    return temp_df_weekday

 
def feat_eng_day(dataframe):
    temp_df_day = (
                dataframe
                .loc[:, ['median_length', 'median_delay', 'median_speed_kmh', 'total_records', 'median_level', 'street', 'day']]
                .groupby(['street', 'day'])
                .agg(['mean', 'median', 'std', 'count'])
                )

    temp_df_day = temp_df_day.fillna(0)
    temp_df_day.columns = [f'{col[0]}__{col[1]}' for col in temp_df_day.columns]
    temp_df_day = temp_df_day.reset_index()
    
    value_cols = list(temp_df_day.columns)
    value_cols.remove('day')
    value_cols.remove('street')

    temp_df_day = temp_df_day.pivot(index='street', columns='day', values=value_cols)
    temp_df_day = temp_df_day.fillna(0)
    temp_df_day.columns = [f'{col[0]}__day_{col[1]}' for col in temp_df_day.columns]
    
    return temp_df_day


def merge_features(dataframe, street_bogor=street_bogor):
    temp_df = feat_eng_numerical(dataframe)
    temp_df_weekday = feat_eng_weekday(dataframe)
    temp_df_day = feat_eng_day(dataframe)
    
    merged_street_bogor = street_bogor.drop('geometry', axis=1).merge(temp_df, how='left', left_on='street', right_index=True)
    merged_street_bogor = merged_street_bogor.merge(temp_df_weekday, how='left', left_on='street', right_index=True)
    merged_street_bogor = merged_street_bogor.merge(temp_df_day, how='left', left_on='street', right_index=True)
    merged_street_bogor = merged_street_bogor.fillna(0)
    merged_street_bogor = merged_street_bogor.drop_duplicates()
    
    float64_cols = merged_street_bogor.select_dtypes('float64').columns
    merged_street_bogor[float64_cols] = merged_street_bogor[float64_cols].astype('float32')
    
    return merged_street_bogor

In [123]:
feat_eng_jams_early_morning = merge_features(jams_early_morning)
feat_eng_jams_morning = merge_features(jams_morning)
feat_eng_jams_afternoon = merge_features(jams_afternoon)
feat_eng_jams_late_afternoon = merge_features(jams_late_afternoon)
feat_eng_jams_night = merge_features(jams_night)
feat_eng_jams_late_night = merge_features(jams_late_night)

In [124]:
feat_eng_jams_early_morning.to_parquet('..\\dataset\\feat_eng_jams_early_morning.parquet')
feat_eng_jams_morning.to_parquet('..\\dataset\\feat_eng_jams_morning.parquet')
feat_eng_jams_afternoon.to_parquet('..\\dataset\\feat_eng_jams_afternoon.parquet')
feat_eng_jams_late_afternoon.to_parquet('..\\dataset\\feat_eng_jams_late_afternoon.parquet')
feat_eng_jams_night.to_parquet('..\\dataset\\feat_eng_jams_night.parquet')
feat_eng_jams_late_night.to_parquet('..\\dataset\\feat_eng_jams_late_night.parquet')